In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

# 0) Load
df = pd.read_csv("pricing_treatment.csv", parse_dates=["date"])
df.columns = df.columns.str.lower()
assert {"date","trade_code","price","bookings"}.issubset(df.columns)

# Basic cleaning
df = df[(df["price"] > 0) & (df["bookings"] >= 0)].copy()

# 1) Optional demand shifter
has_searches = "searches" in df.columns
if has_searches:
    df["ln_searches"] = np.log(df["searches"].clip(lower=0) + 1)

# 2) Calendar controls (lightweight, avoids huge daily FE)
df["week"] = df["date"].dt.isocalendar().week.astype(int)
df["dow"]  = df["date"].dt.dayofweek.astype(int)

# 3) Log(price)
df["ln_price"] = np.log(df["price"])

# 4) (Optional) provide your own mapping: trade_to_cluster = {...}
#    If you already have clusters from get_trade_code_clusters(...), just map them here.
#    Fallback: quick KMeans on stable trade features to get 3 clusters (only if you don't have a mapping)
if "cluster" not in df.columns:
    agg = (df.groupby("trade_code")
             .agg(price_mean=("price","mean"),
                  price_std=("price","std"),
                  bookings_mean=("bookings","mean"),
                  bookings_std=("bookings","std"))
             .fillna(0.0))
    X = (agg - agg.mean()) / agg.std(ddof=0).replace(0, 1)
    from sklearn.cluster import KMeans
    labels = KMeans(n_clusters=3, random_state=0, n_init=10).fit_predict(X.values)
    trade_to_cluster = dict(zip(agg.index, labels))
else:
    trade_to_cluster = dict(zip(df["trade_code"], df["cluster"]))

df["cluster"] = df["trade_code"].map(trade_to_cluster).astype(int)

# 5) Estimate PPML per trade: bookings ~ ln_price + calendar + [ln_searches]
def fit_trade_ppml(dft):
    fml = "bookings ~ ln_price + C(week) + C(dow)"
    if has_searches: fml += " + ln_searches"
    try:
        res = smf.glm(fml, data=dft, family=sm.families.Poisson()).fit()
        b = res.params.get("ln_price", np.nan)                       # elasticity
        se = res.bse.get("ln_price", np.nan)
        p  = res.pvalues.get("ln_price", np.nan)
        ci_lo, ci_hi = res.conf_int().loc["ln_price"].tolist()
    except Exception:
        b, se, p, ci_lo, ci_hi = [np.nan]*5
    return b, se, p, ci_lo, ci_hi

rows = []
for t, dft in df.groupby("trade_code"):
    b, se, p, lo, hi = fit_trade_ppml(dft)
    rows.append({
        "trade_code": t,
        "cluster": trade_to_cluster[t],
        "elasticity": b,    # d ln(bookings)/d ln(price)
        "se": se,
        "p_value": p,
        "ci_low": lo,
        "ci_high": hi,
        "n_obs": len(dft),
        "avg_bookings": dft["bookings"].mean(),
        "avg_price": dft["price"].mean()
    })

trade_elast = (pd.DataFrame(rows)
               .sort_values(["cluster","trade_code"])
               .reset_index(drop=True))

# 6) Aggregate to clusters (inverse-variance weighted mean)
def pooled(g):
    g = g.dropna(subset=["elasticity","se"])
    g = g[g["se"] > 0]
    if len(g) == 0:
        return pd.Series({"elasticity": np.nan, "se": np.nan, "n_trades": 0})
    w = 1.0 / (g["se"]**2)
    beta = (w * g["elasticity"]).sum() / w.sum()
    se   = np.sqrt(1.0 / w.sum())
    return pd.Series({"elasticity": beta, "se": se, "n_trades": len(g)})

cluster_elast = trade_elast.groupby("cluster").apply(pooled).reset_index()
cluster_elast["ci_low"]  = cluster_elast["elasticity"] - 1.96*cluster_elast["se"]
cluster_elast["ci_high"] = cluster_elast["elasticity"] + 1.96*cluster_elast["se"]

print("\n=== Trade-level elasticities (PPML) ===")
print(trade_elast.head(10))
print("\n=== Cluster-level pooled elasticities ===")
print(cluster_elast)



=== Trade-level elasticities (PPML) ===
  trade_code  cluster  elasticity        se       p_value    ci_low   ci_high  \
0         B2        0   -0.050340  0.032959  1.266673e-01 -0.114938  0.014257   
1         D5        0   -0.061708  0.034927  7.726445e-02 -0.130164  0.006747   
2         F7        0   -0.150084  0.030425  8.102072e-07 -0.209716 -0.090452   
3         L4        0   -0.060763  0.026624  2.247373e-02 -0.112945 -0.008581   
4         M5        0   -0.064983  0.025492  1.079754e-02 -0.114946 -0.015020   
5         O0        0    0.038351  0.027536  1.636828e-01 -0.015617  0.092320   
6         A1        1    0.276548  0.042687  9.265342e-11  0.192883  0.360214   
7         A6        1   -0.106865  0.042482  1.188503e-02 -0.190128 -0.023602   
8         C4        1    0.185001  0.040431  4.746887e-06  0.105757  0.264245   
9         D9        1   -0.054963  0.035425  1.207700e-01 -0.124394  0.014468   

   n_obs  avg_bookings    avg_price  
0    180    326.933333  4706.

C:\Users\as26a\AppData\Local\Temp\ipykernel_21124\294748374.py:90: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cluster_elast = trade_elast.groupby("cluster").apply(pooled).reset_index()
